In [ ]:
!pip install rank_bm25

In [ ]:
!pip install --upgrade numexpr

In [ ]:
from rank_bm25 import BM25Okapi
import csv
import pandas as pd
import string


In [ ]:
recurso_topicos = pd.read_csv('bertopic_nao_supervisionado.csv')

## **Prepara corpus de temas repetitivos e inicia BM25**


In [ ]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

In [ ]:
temas_repetitivos = pd.read_csv('temas_repetitivos.csv', sep=',' )
corpus = []
for indice, linha in temas_repetitivos.iterrows():
    temas = linha['tema']
    tema_clean = remove_punctuation(temas)
    corpus.append(tema_clean)
    
tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def sort_list(lista):
    return(sorted(lista, key = lambda x: x[1],reverse=True))

In [ ]:

def create_list_similarity_bm25(temas ,k,tema_real):
  lista_similaridade = temas
  lista_tema_real = []

  sorted_list = sort_list(lista_similaridade)

  for i, linha in enumerate(sorted_list):
      if(linha[0]==tema_real):
        lista_tema_real.append(i+1)#identifica posição do tema real no ranking
        lista_tema_real.append(linha[1])#similaridade do tema real segundo bm25
        break
  ranking = sorted_list[:k]
  return ranking,lista_tema_real

In [ ]:
def create_columns(k):
  #k refere-se ao numero de elementos no ranking
  colunas = []
  colunas.append("indice")
  colunas.append("num_tema_cadastrado")
  for i in range(1, k + 1):
    nome = f"sugerido_{i}"
    colunas.append(nome)
    nome = f"similaridade_{i}"
    colunas.append(nome)
  colunas.append("posicao_tema_real")
  colunas.append("similaridade_tema_real")
  return colunas

In [ ]:
def calc_scores_bm25(topico):
    query = topico.replace("-","")
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    return doc_scores

## **Armazenamento de dados classificados**

In [ ]:
#Cria dataframe pra armazenar dados sobre textos classificados
nomes_colunas = create_columns(6)
resp_classificados_bertopic_bm25 = pd.DataFrame(columns=nomes_colunas)
#Cria arquivo pra armazenar resultados
resp_classificados_bertopic_bm25.to_csv('resp_classificados_bertopic_bm25.csv', index=False)

In [ ]:
#print(recurso_topicos.iloc[0]['topicos'].replace("-",""))
 

In [ ]:
#Tratamento topicos de cada documento e lista de temas
for indice, linha in recurso_topicos.iterrows():
  #calcula score bm25
  doc_scores = calc_scores_bm25(linha['topicos'])
  
  #Cria lista de tuplas com numeração do tema e valor de similaridade
  temas_classificados = temas_repetitivos[['num_tema_cadastrado']].copy()
  temas_classificados['similaridade']=doc_scores
  temas_classificados.columns = ['numTema','similaridade']
  list_temas_classificados = list(temas_classificados.itertuples(index=False, name=None))

  dados = []
  dados.append(indice)
  #numero do tema cadastrado por um analista
  try:
      #numeracao de tema cadastrado pelo analista para o recurso
      dados.append(int(linha['num_tema_cadastrado']))
  except Exception as erro:
      print(f"Erro ao capturar numero de tema cadastrado {indice}")
      continue
  
  try:  
      ranking , lista_tema_real = create_list_similarity_bm25( list_temas_classificados, 6, linha['num_tema_cadastrado'])
  except Exception as erro:
      print(f"Erro calculo similaridade indice {indice}") 
      continue
  for i, tupla_num_tema in enumerate(ranking):
    #captura numero do tema sugerido e valor da similaridade
    dados.append(tupla_num_tema[0])
    dados.append(tupla_num_tema[1])
  
  if(lista_tema_real):
    dados.append(lista_tema_real[0])
    dados.append(lista_tema_real[1])
  else:
    dados.append("NA")
    dados.append("NA")
  #with open('resp_classif_bertopic_unsuperv.csv', mode='a', newline='') as arquivo:
  with open('resp_classificados_bertopic_bm25.csv', mode='a', newline='') as arquivo:
    writer = csv.writer(arquivo)
    writer.writerow(dados)

In [ ]:
#print(tokenized_query)

In [ ]:

#bm25.get_top_n(tokenized_query, corpus, n=1)

In [ ]:
#print(doc_scores)